In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


#This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
import re

# from collections import defaultdict
import pandas as pd
from collections import Counter, defaultdict
# import numpy as np
# #import xmltodict
# import networkx as nx
# from sqlalchemy import text


repo_dir = os.path.split(os.getcwd())[0]

if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# import sys
# !{sys.executable} -m pip install -U jupyter_nbextensions_configurator

In [23]:
# let's repress the warnings, esp. from pandas
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [7]:
import run_attendancelist

In [25]:
runner1728 = run_attendancelist.RunAll(year=1728)


logging to attendancelist.log


In [26]:
%time runner1728.initial_find()

1. finding presidents
286 found
2.find provincial extraordinaris gedeputeerden
CPU times: user 1min 10s, sys: 375 ms, total: 1min 11s
Wall time: 1min 11s


In [27]:
searchobs = runner1728.searchobs

In [28]:
# dir(runner1744)
%time runner1728.delegates_from_fragments()

CPU times: user 3min 18s, sys: 1.17 s, total: 3min 19s
Wall time: 3min 22s


In [29]:
runner1744.find_unmarked_text()

3. finding unmarked text


[['vier',
  "'met",
  'uier',
  'hier',
  'niet',
  'Viet',
  'Miel',
  'Meel',
  'mest',
  'miet',
  'wier',
  'met'],
 ['1744', '1734', '1741', '17144'],
 ['1744Bentinck'],
 ['Acte'],
 ['Agent'],
 ['Ban1744'],
 ['Bentinck'],
 ['Bergsma'],
 ['Bit'],
 ['Bottestein1744',
  'Bottesteun',
  'Botteslein',
  'Buttesten',
  'Bottehitein',
  'BottesteinVan',
  'Boitestein',
  'Bottesten',
  'Bottesiein',
  'Bottcstein',
  'Doitestein',
  'Botiestein',
  'Botrenstein',
  'Bottestin',
  'Bottenstein',
  'Bottessein',
  'Bottesteyn',
  'Boitesteun',
  'Bostestein',
  'Boftestein',
  'Boittestein',
  'Bottestein',
  'Buttestein',
  'Boltestein',
  'BoiTestein'],
 ['Haogendorp',
  'Hogendorp',
  'Bogendorp',
  'Hosendorp',
  'Hogenderp',
  'Ingendorp',
  'fhgendorp'],
 ['Bolch'],
 ['Bop'],
 ['BottesteinBergsma'],
 ['Brantzenburgb',
  'Brantzenborghy',
  'Bruntzenborgh',
  'Brantzenborghb',
  'Bruntzenborgb',
  'Brantzenborgbo',
  'Brauntzenborgb',
  'Braazenborgh',
  'Branizenborgb',
  'Brauzenbor

In [6]:
#  an object that points at attendancelists. 
# Note that for use in comparisons that involve typechecking for strings
# we may have to take a roundtrip converting it to a real string and later retrieve the actual object, but that's fine
# note that for performance reasons we first gather all rfeerences to a searchobject and then add spans 

from collections import UserString
class StringWithContext(UserString):
    def __init__(self, item='', reference=None):
        self.data = item
        self.references = []
        
    def set_reference(self, reference):
        self.references.append(reference)
        
    def get_references(self):
        return self.references



In [7]:
# test

x = StringWithContext('bla')
type(x)
x.lower()
x.set_reference('q23')
'q23' in x.references




True

### obs_to_fragments

In [8]:
# gather unmarked snippets
# group them, with references to days by using the stringwithcontext object
import re
found = run_attendancelist.found_delegates
df = run_attendancelist.abbreviated_delegates
i_s = run_attendancelist.iterative_search
noresults = []

for ob in searchobs:
    so = searchobs[ob].matched_text
    jitems = ';'.join(so.get_unmatched_text())
    items = [item.strip() for item in re.split('[;,\.]+', jitems) if item.strip()!='' and len(item.strip())>2]
    for item in items:
        contextitem = StringWithContext(item)
        contextitem.set_reference(so)
        noresults.append(contextitem)
        

In [9]:
#these are all unmarked spans in the attendance lists
#probably we best sort these by hand and try and assign them 
# Counter([i for i in noresults]).most_common()
noresultscounter = Counter(noresults)

In [11]:
from run_attendancelist import FuzzyKeywordGrouper
from numpy import argmax

In [12]:
# gather most frequent non-identified groups and 
# assign them the most frequent name as key, under the assumption that the most frequent pattern is usually right
unmatched_grouped = FuzzyKeywordGrouper([str(k) for k in noresultscounter.keys()]).vars2graph()
consolidated_groups = {}
for group in unmatched_grouped:
    i = argmax([noresultscounter[i] for i in group])
    consolidated_groups[group[i]] = [g for g in group if g != group[i]]
        
consolidated_groups

{'met vyf': ['met eén',
  'met eenu',
  'met vwys',
  'met vys',
  'met cen',
  "'met vyf",
  'met wyf',
  'met ses'],
 'Van Tamminga': ['Van ZTamminge',
  'Vun Tamminga',
  '-Van Zamminga',
  "P'an Tamminga",
  'Vlan Tamminga',
  'Wan Tamminga',
  'Van ZTamminga',
  'Van JTamminga',
  'Vun Zamminga',
  "J'an Tamminga",
  'Van Zamminga'],
 '17144 DE Resosutien gisteren genomen': ['1744 DE Resorditieen ginteren geno men',
  'OverysselDE Resolutien gisteren genomen',
  'OverWESDE Resoluten gisteren genomen'],
 '1734 Bentinck': ['1744 Bentinck', '1744Bentinck'],
 '1744 Bergsma': [],
 '1744 Heeren Gecommitteerde Raaden van de pypvintie van Holland en Westvriesland': [],
 'Verelst Van Haren': ['1744 Van Haren', 'EE A Van Haren'],
 '1744 Vun Utenhove tot Boitestein': [],
 '1744 neevens die welcke gevoeght is geweest by een Missive van den Secretaris Bolch': [],
 '1744 plint van meeninge was': [],
 'Ingilby': ['Ingiloyv',
  'lrgilby',
  'JIngelby',
  'lr gzilby',
  'JIngilby',
  'si gilby',
 

In [13]:
from republic.helper.utils import reverse_dict
r_cons = reverse_dict(consolidated_groups)

### consolidated counter

In [14]:
consolidated_counter = defaultdict(int)
for key in noresultscounter:
    nkey = r_cons.get(key) or key
    consolidated_counter[nkey] += noresultscounter[key]

## Problem: concatenated strings
There are items that are part of either a date or a resumption, which means they did not get recognized as part of a resumption of some other sort of spurious text, which means the parsing did not go right
We have items that are concatenated. Can we split these.

## Strategy for identifying unknown spans

- run regular checks
- gather unmarked snippets
- group them, with references to days (how?)
- gather most frequent non-identified groups
    - can we identify them?
    - if we can, apply identification to references
    - if not, make unidentified references (which id?) and add it to the list of references
- how many unidentified spans have we still got left
- filter out spurious words (frequency?)
- how many are there still. Do they look like names? (begin with 'van' pat, capitals, are they part of composite names, that we have identified)

further processing:

- do we have double identification that may be reduced, because they are part of names
- after that:
    - are there any outlying presidents: that occur only once or out of place (in a differently identified row)
    - can we do something with the order of delegates

### preliminary step - identify junk

In [15]:
from republic.data.stopwords import stopwords
from fuzzy_search.fuzzy_string import score_levenshtein_similarity_ratio
import itertools
import regex

In [16]:
from run_attendancelist import junksweeper
js = junksweeper
suspected_items = []
numpat = re.compile('[0-9]{2,}')
for item in consolidated_groups.keys():
    if numpat.search(item):
        suspected_items.append(item)
#     elif js.find_matches(item):
#         suspected_items.append(item)        
    # elif not prefixpat.search(item.split(' ')[0]) and item[0] in string.ascii_lowercase:
    #     suspected_items.append(item)
    elif item == item.lower(): # no capitals means no name
        suspected_items.append(item)
    elif item == item.upper():
        suspected_items.append(item)
suspected_items 

['met vyf',
 '17144 DE Resosutien gisteren genomen',
 '1734 Bentinck',
 '1744 Bergsma',
 '1744 Heeren Gecommitteerde Raaden van de pypvintie van Holland en Westvriesland',
 '1744 Vun Utenhove tot Boitestein',
 '1744 neevens die welcke gevoeght is geweest by een Missive van den Secretaris Bolch',
 '1744 plint van meeninge was',
 'A G',
 'DE Re oren Ii B NESen ge15',
 'Hp» 1741 van Cattenburch',
 'Lunae den 14 September1744PRAESIIDE',
 'Lunae den 25 Mey1744',
 'Martis den 26 Mey1744',
 'met',
 'PRAAESEHENTIPUS',
 'PRADESENTIEBUS',
 'PRAESITDE',
 'PRAEZESENTITIEJS',
 'PROUOESIDE',
 'TIBWUXS',
 'TIEBEUS',
 'TIPBPIUS)',
 'TITIBUS',
 'tenburch',
 'Van Utenhove tot Bottestein1744 Van Haaren',
 'door den Raad van Staate1744',
 'en verstaan',
 'en vervolgens',
 'en vervolgens weder te rugge',
 'en wederom herwaarts te moogen doen',
 'is goedgevonden en verstaan',
 'lonel',
 'mei',
 'mei tuu',
 'men',
 'met den druck gemeen gemaackt sullen werden',
 'met een extrau',
 'met een extraordinaris Ge1

#### comment

junk spotting worked well, but we still have some items that contains junk with 

In [17]:
import regex
def name_matcher(naam1,naam2, threshold):
    result = False
    n1, n2 = sorted([naam1, naam2], key=lambda x: len(x))
    pat = regex.compile('(?P<otherpart>.*)(?P<namepart>' + regex.escape(n1) + '){e<=2}' + '(?P<lastpart>.*)', flags=regex.BESTMATCH)
    r = pat.search(n2)
    if r:
        np = r.groupdict()['namepart']
        s = score_levenshtein_similarity_ratio(np, n1)
        if s > threshold:
            result = r.groupdict()
            result['similarity'] = s
    return result

In [18]:
singlekeys = [k for k in consolidated_groups if consolidated_counter.get(k)==1]
to_compare = list(itertools.product(list(consolidated_groups.keys()), singlekeys))

In [19]:
tc2 = list(itertools.combinations(list(consolidated_groups.keys()), 2))

In [20]:
len(to_compare) > len(tc2)

True

In [21]:
print(len(to_compare), len(tc2))

25773 22578


In [22]:

scores2 = []
for key, cand in to_compare: #itertools.combinations(list(consolidated_groups.keys()), 2):
    literal = False
    sc = name_matcher(key, cand, threshold=0.7)
    key, cand = sorted([key, cand], key=lambda x: len(x))
    if sc:
        if key not in suspected_items and cand not in suspected_items:
            scores2.append({'orig_key': cand,
                            'sub_pattern1': key,
                            'similarity': sc['similarity'],
                            'pre_pattern':sc['otherpart'],
                            'trailing_pattern': sc['lastpart']})

In [23]:
keyframe = pd.DataFrame.from_records(scores2)

In [24]:
keyframe

,orig_key,sub_pattern1,similarity,pre_pattern,trailing_pattern
0,OverysselVan Tamminga,Van Tamminga,1.000000,Overyssel,
1,met een extraordinaris Gedeputeerde uyl de Pro...,Van Tamminga,1.000000,met een extraordinaris Gedeputeerde uyl de Pro...,
2,Fabricius Ingelby,Ingilby,0.857143,Fabricius,
3,en aan hem ten respecte van haar Hoogh Mogende...,Van Laar,0.750000,en aan hem ten respecte v,Hoogh Mogende Caracter door een Koninghlijcke...
4,weshalven hy onder het welneemen van haar Hoog...,Van Laar,0.750000,weshalven hy onder het welneemen v,Hoogh Mogende hem daar omtrent geconformeert ...
...,...,...,...,...,...
237,versoeckende den Suppliant PRAESIDE,Over,0.750000,,soeckende den Suppliant PRAESIDE
238,versoeckende den Suppliant PRAESIDE,versoeckende den Suppliant PRAESIDE,1.000000,,
239,was gebraght in de Classis van vyftien honderd...,was gebraght in de Classis van vyftien honderd...,1.000000,,
240,weshalven hy onder het welneemen van haar Hoog...,dat haar Hoogh Mog,0.888889,weshalven hy onder het welneemen v,ende hem daar omtrent geconformeert hadde


In [25]:
multiple_recog = keyframe.orig_key.value_counts()>1

In [26]:
len(multiple_recog)

89

In [27]:
ambiv = list(multiple_recog.index)

In [28]:
keyframe.loc[keyframe.orig_key.isin(ambiv)]

,orig_key,sub_pattern1,similarity,pre_pattern,trailing_pattern
0,OverysselVan Tamminga,Van Tamminga,1.000000,Overyssel,
1,met een extraordinaris Gedeputeerde uyl de Pro...,Van Tamminga,1.000000,met een extraordinaris Gedeputeerde uyl de Pro...,
2,Fabricius Ingelby,Ingilby,0.857143,Fabricius,
3,en aan hem ten respecte van haar Hoogh Mogende...,Van Laar,0.750000,en aan hem ten respecte v,Hoogh Mogende Caracter door een Koninghlijcke...
4,weshalven hy onder het welneemen van haar Hoog...,Van Laar,0.750000,weshalven hy onder het welneemen v,Hoogh Mogende hem daar omtrent geconformeert ...
...,...,...,...,...,...
237,versoeckende den Suppliant PRAESIDE,Over,0.750000,,soeckende den Suppliant PRAESIDE
238,versoeckende den Suppliant PRAESIDE,versoeckende den Suppliant PRAESIDE,1.000000,,
239,was gebraght in de Classis van vyftien honderd...,was gebraght in de Classis van vyftien honderd...,1.000000,,
240,weshalven hy onder het welneemen van haar Hoog...,dat haar Hoogh Mog,0.888889,weshalven hy onder het welneemen v,ende hem daar omtrent geconformeert hadde


### steps from above
-  How to make sure we have the best pattern?
- The best is when the remaining part is also a key (14 cases),though not all keys are ok


In [29]:

ks = [ key for key in consolidated_groups.keys() if key not in suspected_items]
both_keys = keyframe.loc[keyframe.trailing_pattern.isin(ks)]
print(len(both_keys))
both_keys # these are ok

6


,orig_key,sub_pattern1,similarity,pre_pattern,trailing_pattern
47,vander HeimMogge van Renesse,Heim,1.000000,vander,Mogge van Renesse
74,OverysselVan Tamminga,Overysel,0.888889,,Van Tamminga
75,met een extraordinaris Gedeputeerde uyl de Pro...,Overysel,0.888889,met een extraordinaris Gedeputeerde uyl de Pro...,Van Tamminga
105,van JismaGerlacius,van Itsma,0.777778,,Gerlacius
148,vander HeimMogge van Renesse,vander Heim,1.000000,,Mogge van Renesse
234,vander HeimMogge van Renesse,Heim,1.000000,vander,Mogge van Renesse


In [30]:
def_keys_with_subPatterns = [{'key': x[0], x[1]: x[1], x[2]:x[2]} for x in 
                               both_keys[['orig_key','sub_pattern1','trailing_pattern']].to_records(index=False)]

In [31]:
# start_prefixpat = re.compile(r"^[à|à|d\'|de|den|der|des|di|en|het|in 't|la|la|le|of|van|ten|thoe|tot|'t|Heeren|van|met|Holland]",
# re.UNICODE)

def match_ppat(x):
    result = False
    if prefixpat.search(x):
        result = True
    return result

In [33]:
def levntest(x, lijst):
    result = 0
    for z in lijst:
        r = score_levenshtein_similarity_ratio(x, z)
        if r> result:
            result = r
    return result

In [34]:
from IPython.display import display
#- If we have several 'legitimate' keys, perhaps score on the exactness of recognition? Or the longest? Or a combination
grkeyframe = keyframe.groupby('orig_key')

for group in grkeyframe.groups:
    subfr = grkeyframe.get_group(group)
    vals = subfr.sub_pattern1.unique()
    subfr['tr'] = subfr.trailing_pattern.apply(lambda x: levntest(x, vals))
    subfr['pr'] = subfr.pre_pattern.apply(lambda x: levntest(x, vals))
    deffr = subfr.loc[(subfr.tr > 0.75) | (subfr.pr > 0.75)]
    if len(deffr)>0:
        pre = deffr.loc[(deffr.tr==deffr.tr.max())]
        post = deffr.loc[deffr.pr==deffr.pr.max()]
        result = {'key': group, pre.sub_pattern1.iloc[0]:post.pre_pattern.iloc[0], 
                  post.sub_pattern1.iloc[0]:pre.trailing_pattern.iloc[0]}
        def_keys_with_subPatterns.append(result)
    else:
        deffr = subfr.loc[subfr.similarity> 0.75]
        if len(deffr)>0:
            result = {'key': group, 
                      subfr.sub_pattern1.iloc[0]: subfr.sub_pattern1.iloc[0],
                      subfr.pre_pattern.iloc[0]: subfr.pre_pattern.iloc[0],
                      subfr.trailing_pattern.iloc[0]: subfr.trailing_pattern.iloc[0]}
            def_keys_with_subPatterns.append(result)
            #display(deffr)

In [36]:

def is_this_junk(item):
    result = False
    if item=='':
        result = True
    elif numpat.search(item):
        result = True
    elif js.find_matches(item):
        result = True
    # elif not prefixpat.search(item.split(' ')[0]) and item[0] in string.ascii_lowercase:
    #     suspected_items.append(item)
    elif item == item.lower(): # no capitals means no name
        result = True
    elif item == item.upper():
        result = True
    return result

In [37]:
itj = is_this_junk
#test
item = 'genomen'
result_itj = item == item.lower()
print(result_itj)
result_itj = itj(item)
print(result_itj)

True
True


In [38]:
def_keys_with_subPatterns

[{'key': 'vander HeimMogge van Renesse',
  'Heim': 'Heim',
  'Mogge van Renesse': 'Mogge van Renesse'},
 {'key': 'OverysselVan Tamminga',
  'Overysel': 'Overysel',
  'Van Tamminga': 'Van Tamminga'},
 {'key': 'met een extraordinaris Gedeputeerde uyl de Provintie vam OverysselVan Tamminga',
  'Overysel': 'Overysel',
  'Van Tamminga': 'Van Tamminga'},
 {'key': 'van JismaGerlacius',
  'van Itsma': 'van Itsma',
  'Gerlacius': 'Gerlacius'},
 {'key': 'vander HeimMogge van Renesse',
  'vander Heim': 'vander Heim',
  'Mogge van Renesse': 'Mogge van Renesse'},
 {'key': 'vander HeimMogge van Renesse',
  'Heim': 'Heim',
  'Mogge van Renesse': 'Mogge van Renesse'},
 {'key': 'Bergsma van lisma', 'van Itsma': ''},
 {'key': 'Bop', 'Bop': 'Bop', '': ''},
 {'key': 'DE NEON g ae ien genomen',
  'DE NEON g ae ien genomen': 'DE NEON g ae ien genomen',
  '': ''},
 {'key': 'DE Ren gsxr en geno',
  'DE Ren gsxr en geno': 'DE Ren gsxr en geno',
  '': ''},
 {'key': 'DE Resolutien voor eergil',
  'DE Resolutien 

In [39]:
nks = [r['key'] for r in def_keys_with_subPatterns]
for g in [g for g in grkeyframe.groups if g not in nks]:
    display(grkeyframe.get_group(g)) # what is left does not count or does not contain

,orig_key,sub_pattern1,similarity,pre_pattern,trailing_pattern
70,Leeuwen,Leen,0.75,,wen


,orig_key,sub_pattern1,similarity,pre_pattern,trailing_pattern
117,de Vos van Steenwyck,Leen,0.75,de Vos van St,wyck


,orig_key,sub_pattern1,similarity,pre_pattern,trailing_pattern
164,met een extraordinaris Gear puleerde uyt de Pr...,Leen,0.75,met,extraordinaris Gear puleerde uyt de Provintie...


,orig_key,sub_pattern1,similarity,pre_pattern,trailing_pattern
165,met een extraordinaris Gede,Leen,0.75,met,extraordinaris Gede


In [40]:
origkeys = list(consolidated_groups.keys())

In [41]:
from republic.helper.utils import reverse_dict
r_cons = reverse_dict(consolidated_groups)

### groups to references

In [42]:
consolidated_counter = defaultdict(int)
for key in noresultscounter:
    nkey = r_cons.get(key) or key
    consolidated_counter[nkey] += noresultscounter[key]

In [48]:
nwrecords = defaultdict(list)
quarantine = []
xx = xxx = []
nokeys = [str(n) for n in noresults]
for record in def_keys_with_subPatterns:
    for i in record:
        if i == 'key':
            quarantine.append(record[i])
        else:
            aant = consolidated_counter.get(i) or 0
            if aant > 1:
                if good_key(i):
                    xx.append(i)
                    refs = [ob.references for ob in 
                        list(filter(lambda x: x == i, noresults)) if len(ob.references)>0][0]
                    nwrecords[i].extend(refs)


In [47]:
prefixpat = re.compile(r"à\b|\bà\b|\bd\'\b|\bde\b|\bden\b|\bder\b|\bdes\b|\bdi\b|\ben\b|\bhet\b|\bin 't\b|\bla\b|\bla\b|\ble\b|of|van|\bten\b|\bthoe\b|\btot\b|\b't\b|\bHeeren\b|\bvan\b|\bmet\b|\bHolland", flags =  re.I)
           #r'\b|\b'.join(stopwords), flags =  re.I)

In [44]:
 
def looks_like_a_name(pat):
    if pat == '':
        return False
    r = False
    pats = pat.split(' ')
    pats = [pat for pat in pats if not prefixpat.search(pat)  ]
    # starts with a prefix
    # if prefix:
    #     pats = pats[1:]
    for p in pats:
        if p.capitalize() == p:
            r = True
        else:
            r = False
            break
    return r

In [58]:
pat = 'Beur'
pats = pat.split(' ')
pats = [pat for pat in pats if not prefixpat.search(pat)  ]
for p in pats:
        if p.capitalize() == p:
            r = True
            break
        else:
            r = False
r

True

In [45]:
def good_key(pat):
    r = True
    #print(pat, r)
    if itj(pat):
        #print('junk',pat,r)
        r = False
    else:
        if not looks_like_a_name(pat):
            #print('name',pat,r)
            r = False 
    return r

In [54]:
rejected_keys = []
xgroups = {}
for g in consolidated_groups:
    if str(g) not in quarantine:
        if not itj(str(g)):
            ks = [g] + consolidated_groups[g]
            r = list(filter(lambda i: i in ks, noresults))
            ng = defaultdict(list)
            for key in list(set([str(i) for i in r])):
                refs = filter(lambda x: x == key, r)
                chrefs = list(itertools.chain([x.references for x in refs]))
                ng[key].extend(list(itertools.chain.from_iterable(chrefs)))
            xgroups[g] = ng
        else:
            rejected_keys.append(g)

In [55]:
for item in  nwrecords:
    if xgroups.get(item):
        xgroups[item][item].extend(nwrecords[item])
    else:
        rejected_keys.append(item)


In [58]:
found = run_attendancelist.found_delegates
df = run_attendancelist.abbreviated_delegates
i_s = run_attendancelist.iterative_search

In [59]:
vanpat = re.compile('vande[rn]?|([d|t]+e?[rn]?)\s+|van', re.I)
#for phrase in all_phrases:
#     print(phrase, [x.strip() for x in vanpat.split(phrase) if x and x.strip() !=''])

In [60]:

def is_namepart(variant, found=found):
    result = {}
    result['phrase'] = variant
    result['contained'] = False
    result['exactname'] = False
    match = found.variants.apply(lambda x: levenst_vals(x, phrase))
    matchname = list(found.loc[match].name)[0]
    refid = list(found.loc[match].ref_id)[0]
    result['matchname'] = matchname
    result['id'] = refid
    searchpat = [x.strip() for x in vanpat.split(phrase) if x and x.strip() !='']
    for p in searchpat:
        if p in matchname:
            result['exactname'] = True
            if len(phrase)<len(matchname):
                result['contained'] = True
    return result

In [61]:
def rate_match(key, match):
    return match.levenshtein_similarity * len(match.string)/len(key)

In [62]:
js = run_attendancelist.junksweeper
suspects = []
pat_nonstring = re.compile("\w+[0-9\(\)]+\w+")
for k in xgroups:
    check = False
    key = re.sub(pat_nonstring,'', k)
    # if k!=key:
    #     print(k, "***", key)
    jmatches = js.find_matches(key, include_variants=True, use_word_boundaries=False)
    if len(jmatches)>0:
        bestjmatch = max(jmatches, key=lambda m: rate_match(key,m))
        if len(bestjmatch.string)/len(key) > 0.5: # more than half of the string is in vanpat
            vancheck = vanpat.search(bestjmatch.string)
            if vancheck:            
                if k!=key:
                    key = k
                suspects.append(key)
        print(key, bestjmatch.string, check)
    

Van Laar aar False
Daandels Daand False
Dyckmeester est False
Sirick van LinschotenVan Haaren Haar False
Van Haaren Haar False
Van Rechteren hteren False
de Vos van Steenwyck een False
Zealand land False
van Haarsolte van Jrst Haar False
van Haarsolte Haar False
van Westervelt West False


In [63]:
suspects

[]

In [64]:
levenst_vals = run_attendancelist.levenst_vals

In [65]:
from run_attendancelist import herensearcher, junksweeper, FndMatch
fndmatch = FndMatch(year=1730,
                   searcher=herensearcher,
                   junksearcher=junksweeper,
                   register={},
                   rev_graph={},
                   df=df)
def match_from_dict(pattern, founddb=found, fndmatch=fndmatch):
    """pattern should be string in context"""
    
#     longpats = fs.find_matches(pattern, include_variants=True)
#     if len(longpats)>0:
#         result = max(longpats, key=lambda x: x.levenshtein_similarity)
#     else:
    res = {}
    nm = ''
    idnr = 0
    match = found.variants.apply(lambda x: levenst_vals(x, pattern))
    result = found.loc[match]
    res['pattern'] = pattern
    if len(result) > 0:
        nm = result.name.iat[0]
        idnr = result.id.iat[0]
    else:
        result = fndmatch.match_candidates(heer=keyw).serialize()
        if result['score'] > 0.7:
            lookup = i_s(name=result['match_keyword'], year=1730, df=df)
            if len(lookup)>0:
                nm = lookup.name.iat[0]
                idnr = lookup.id.iat[0]
    res['proposed_delegate'] = nm
    res['p_id'] = idnr
    return res

In [67]:
from run_attendancelist import herensearcher, junksweeper, FndMatch
match_records = {}
fndmatch = FndMatch(year=1730,
                   searcher=herensearcher,
                   junksearcher=junksweeper,
                   register={},
                   rev_graph={},
                   df=df)
for keyw in xgroups:
    if keyw not in suspects:
        r = match_from_dict(keyw, founddb=found)
#     else:
#         r = {}
    
        
    # if r['proposed_delegate']=='':
    #     result = fndmatch.match_candidates(heer=keyw).serialize()
    #     if result['score'] > 0.7:
    #         lookup = i_s(name=result['match_keyword'], year=1730, df=df)
    #         result['proposed_delegate'] = lookup.name.iat[0]
    #         result['p_id'] = lookup.id.iat[0]
    #         r = result
    # return r
    match_records[keyw] = r

In [68]:
# reverse the matches to searchobjects for faster matching
reference_table = defaultdict(list)
for mkey in xgroups.keys():
    grp = xgroups[mkey]
    for key in grp:
        for r in grp[key]:
            rec = {'dlg':match_records[mkey], 'srchpat': key}
            reference_table[r].append(rec)

In [69]:
for ob in reference_table.keys():
    for m in reference_table[ob]:
        p = m['srchpat']
        r = m['dlg']
        s = re.search(re.escape(str(p)), ob.item)
        if s:
            ob.set_span(s.span(), 
                        clas='delegate', 
                        pattern=p,
                        score=levenst_vals(r.get('pattern'), p),
                        delegate_name=r.get('proposed_delegate'),
                        delegate_id=r.get('p_id'))

In [24]:
from IPython.display import HTML
htmlout=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"\n<h4>{T}</h4>\n" + rest
    if url:
        rest += f"""<br/><br/><a href='{url}'>link naar {T}-image</a><br/>"""
    htmlout.append(rest)
#out.reverse()
HTML("<br><hr><br>".join(htmlout))


In [73]:
from IPython.display import HTML
htmlout=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"\n<h4>{T}</h4>\n" + rest
    if url:
        rest += f"""<br/><br/><a href='{url}'>link naar {T}-image</a><br/>"""
    htmlout.append(rest)
#out.reverse()
HTML("<br><hr><br>".join(htmlout))


In [98]:
import random
ks = random.choices(list(runner1730.searchobs.keys()), k=20)
# pres = []
# prs = []
for T in ks:
    ob = runner1730.searchobs[T].matched_text
    print(T)
    for f in ob.get_fragments():
        if f['type']=='delegate':
            print (f)


session-1730-02-01-num-1
{'type': 'delegate', 'idnr': 0, 'pattern': 'van Lynden', 'begin': 90, 'end': 100, 'delegate_id': 18599, 'delegate_name': 'van Lynden', 'delegate_gewest': False, 'delegate_score': 0}
{'type': 'delegate', 'idnr': 0, 'pattern': 'van Welderen', 'begin': 102, 'end': 114, 'delegate_id': 17862, 'delegate_name': 'van Welderen', 'delegate_gewest': False, 'delegate_score': 0}
{'type': 'delegate', 'idnr': 0, 'pattern': 'Torck', 'begin': 116, 'end': 121, 'delegate_id': 19114, 'delegate_name': 'Torck', 'delegate_gewest': False, 'delegate_score': 0}
{'type': 'delegate', 'idnr': 0, 'pattern': 'Van Blocklandt', 'begin': 194, 'end': 208, 'delegate_id': 15268, 'delegate_name': 'Brandwijk van Blokland', 'delegate_gewest': False, 'delegate_score': 0}
{'type': 'delegate', 'idnr': 0, 'pattern': 'vanden Born', 'begin': 211, 'end': 222, 'delegate_id': 13492, 'delegate_name': 'van den Born', 'delegate_gewest': False, 'delegate_score': 0}
{'type': 'delegate', 'idnr': 0, 'pattern': 'van 

# Junkyard

In [133]:

def is_namepart(variant, found=found):
    result = {}
    result['phrase'] = variant
    result['contained'] = False
    result['exactname'] = False
    match = found.variants.apply(lambda x: levenst_vals(x, phrase))
    matchname = list(found.loc[match].name)[0]
    refid = list(found.loc[match].ref_id)[0]
    result['matchname'] = matchname
    result['id'] = refid
    searchpat = [x.strip() for x in vanpat.split(phrase) if x and x.strip() !='']
    for p in searchpat:
        if p in matchname:
            result['exactname'] = True
            if len(phrase)<len(matchname):
                result['contained'] = True
    return result

class flexible_name_matcher(object):
    def __init__(self, name):
        nameparts = name.split(' ')
        self.vanpat = regex.compile('^van(de[rn]?)?|([d|t]+e?[rn]?)|van', re.I)
        self.parts = [regex.escape(n) for n in nameparts if not vanpat.search(n) or vanpat]
    
    def match(self, input):
        results = []
        for part in self.parts:
            try:
                errorrate = math.floor(len(part)/2)
                pattern ='(?P<firstpart>.*)(?P<namepart>' + part + '){e<=' + f'{errorrate}' +'}' + '\s+(?P<lastpart>.*)'
                r = regex.compile(pattern, flags=regex.BESTMATCH)
                result = r.search(input)
                if result:
                    results.append(result.groupdict())
                results.append(result)
            except:
                
        return results
    # vanpat = regex.compile('vande[rn]?|([dt]+e?[rn]?)\s+|van', re.I)
    # n1 = naam1.split(' ')
    # pat = regex.compile('(?P<otherpart>.*)(?P<namepart>' + regex.escape(n1) + '){e<=3}' + '(?P<lastpart>.*)', flags=regex.BESTMATCH)

In [134]:
fnm = flexible_name_matcher('Bors van Waveren')
for item in borssen:
    fnm.match(item))

[{'firstpart': '', 'namepart': 'Bors', 'lastpart': 'van Waveren'}, <regex.Match object; span=(0, 16), match='Bors van Waveren'>, {'firstpart': 'Bors ', 'namepart': 'van', 'lastpart': 'Waveren'}, <regex.Match object; span=(0, 16), match='Bors van Waveren'>, None]
[{'firstpart': '', 'namepart': 'Bors', 'lastpart': 'van HraverenVan Borsselen'}, <regex.Match object; span=(0, 30), match='Bors van HraverenVan Borsselen'>, {'firstpart': 'Bors ', 'namepart': 'van', 'lastpart': 'HraverenVan Borsselen'}, <regex.Match object; span=(0, 30), match='Bors van HraverenVan Borsselen'>, None]
[None, {'firstpart': 'V', 'namepart': 'an', 'lastpart': 'Borsselen'}, <regex.Match object; span=(0, 13), match='Van Borsselen', fuzzy_counts=(0, 0, 1)>, None]
[None, {'firstpart': 'VeckhoV', 'namepart': 'an', 'lastpart': 'Borselen'}, <regex.Match object; span=(0, 18), match='VeckhoVan Borselen', fuzzy_counts=(0, 0, 1)>, None]
[None, {'firstpart': 'VeckhovenV', 'namepart': 'an', 'lastpart': 'Borsselen'}, <regex.Matc

Let's see what is still not marked

In [74]:
# gather unmarked snippets
# group them, with references to days by using the stringwithcontext object
import re
found = run_attendancelist.found_delegates
df = run_attendancelist.abbreviated_delegates
i_s = run_attendancelist.iterative_search
remainingunmarked = []

for ob in searchobs:
    so = searchobs[ob].matched_text
    jitems = ';'.join(so.get_unmatched_text())
    items = [item.strip() for item in re.split('[;,\.]+', jitems) if item.strip()!='' and len(item.strip())>2]
    for item in items:
        contextitem = StringWithContext(item)
        contextitem.set_reference(so)
        remainingunmarked.append(contextitem)
        

In [75]:
remainingunmarked

['van Heeckeren tot den Brantzenborgh',
 'van Hogendorp',
 'vander Heim',
 'van Utenhove tot Bottenstein',
 'Overylsel',
 'DE Resolutien gisteren genomen',
 'van Heeckeren tot den Brantzenborgh',
 'Raaapenhonaris vander Heim',
 'van Utenhove tot Bottestein',
 'DD Resosurtien gisteren zenomen',
 'uary1744PRAESIDE',
 'met ses',
 'vander Heim',
 'Vun Utenhove tot Bottestein',
 'met cen',
 'Overyisel',
 'DE Resoslutien giseeren genomen',
 'met drie extraordinaris Gedeputeerden uyt de Provintie vam Gelderland',
 'vander Heim',
 'van Utenhove tot Bottestein',
 'Overyisel',
 'van Utenbove tot Bottestein',
 'PRAESTITDE',
 'van Utenhove tot Bottestein',
 'OverysselGerlacius',
 'van Utenbove tot Bottestein',
 'DE Resolutien gisteren genomen',
 'van Utenhove tot Bottenstein',
 'DE Resolutien gisteren genomen',
 'van Utenhove tot Bottesten',
 'vander Heim',
 'van Utenhove tot Bottestein',
 'DE esolutien eergiuteren genoomen',
 'van Heeckeren tot den Brantzenborghb',
 'met vier',
 'van Hogendorp',


In [76]:
#these are all unmarked spans in the attendance lists
#probably we best sort these by hand and try and assign them 
# Counter([i for i in noresults]).most_common()
rresultscounter = Counter(remainingunmarked)

In [77]:
#The raw counts of the results are as follows
rresultscounter.most_common()

[('vander Heim', 200),
 ('van Hogendorp', 104),
 ('van Heeckeren tot den Brantzenborgh', 75),
 ('van Heeckeren tot Roderlo', 67),
 ('DE Resolutien gisteren genomen', 63),
 ('Ingilby', 60),
 ('Ingelby', 47),
 ('van Heeckeren tot den Brantzenborgb', 33),
 ('van Utenhove tot Bottestein', 32),
 ('Van Utenhove tot Bottestein', 25),
 ('Holland en Westvriesland', 22),
 ('met vier', 21),
 ('wan Heeckeren tot Roderlo', 20),
 ('Gelderlund', 16),
 ('met vyf', 12),
 ('wan Heeckeren tot den Brantzenborgh', 10),
 ('DE Resosutien gisteren genomen', 9),
 ('DE Nesolutien gisteren genomen', 9),
 ('wander Heim', 9),
 ('met cen', 8),
 ('van Utenbove tot Bottestein', 8),
 ('van Heeckeren tot den Brant zenborgh', 8),
 ('Overysel', 8),
 ('van Utenhove tot Bottesten', 7),
 ('met', 7),
 ('wan Heeckeren tot den Brantzenborgb', 6),
 ('Ingilty', 5),
 ('van Utenhove tot Bottenstein', 4),
 ('Overyisel', 4),
 ('van Hecckeren tot den Brantzenborgh', 4),
 ('DE Nesosutien gisteren genomen', 4),
 ('wan Hogendorp', 4),
 

In [78]:
# gather most frequent non-identified groups and 
# assign them the most frequent name as key, under the assumption that the most frequent pattern is usually right
runmatched_grouped = FuzzyKeywordGrouper([str(k) for k in rresultscounter.keys()]).vars2graph()
rconsolidated_groups = {}
for group in runmatched_grouped:
    i = argmax([rresultscounter[i] for i in group])
    rconsolidated_groups[group[i]] = [g for g in group if g != group[i]]
        
rconsolidated_groups

{'met vyf': ['met eén',
  'met eenu',
  'met vwys',
  'met vys',
  'met cen',
  "'met vyf",
  'met wyf',
  'met ses'],
 '17144 DE Resosutien gisteren genomen': ['1744 DE Resorditieen ginteren geno men',
  'OverysselDE Resolutien gisteren genomen',
  'OverWESDE Resoluten gisteren genomen'],
 '1734 Bentinck': ['1744 Bentinck', '1744Bentinck'],
 '1744 Bergsma': [],
 '1744 Heeren Gecommitteerde Raaden van de pypvintie van Holland en Westvriesland': [],
 '1744 Vun Utenhove tot Boitestein': [],
 '1744 neevens die welcke gevoeght is geweest by een Missive van den Secretaris Bolch': [],
 '1744 plint van meeninge was': [],
 'Ingilby': ['Ingiloyv',
  'lrgilby',
  'JIngelby',
  'lr gzilby',
  'JIngilby',
  'si gilby',
  'Ingelby',
  'Ingelly',
  'Ingelhy',
  'Ingilty',
  'TIngelby',
  'In gelby',
  'Iugelby',
  'gilby',
  'Inilby',
  '1ngilby',
  'Ingilôy',
  'Inzilby',
  'h gilby'],
 'A G': [],
 'Bergsma van lisma': [],
 'Holland en Westvriesland': ['Holland en Yestvriesland',
  'Bolland en Wees

In [80]:
from run_attendancelist import junksweeper
js = junksweeper
rsuspected_items = []
numpat = re.compile('[0-9]{2,}')
for item in rconsolidated_groups.keys():
    if numpat.search(item):
        rsuspected_items.append(item)
    elif js.find_matches(item):
         rsuspected_items.append(item)
        
    # elif not prefixpat.search(item.split(' ')[0]) and item[0] in string.ascii_lowercase:
    #     suspected_items.append(item)
    elif item == item.lower(): # no capitals means no name
        rsuspected_items.append(item)
    elif item == item.upper():
        rsuspected_items.append(item)
rsuspected_items 

['met vyf',
 '17144 DE Resosutien gisteren genomen',
 '1734 Bentinck',
 '1744 Bergsma',
 '1744 Heeren Gecommitteerde Raaden van de pypvintie van Holland en Westvriesland',
 '1744 Vun Utenhove tot Boitestein',
 '1744 neevens die welcke gevoeght is geweest by een Missive van den Secretaris Bolch',
 '1744 plint van meeninge was',
 'Ingilby',
 'A G',
 'Holland en Westvriesland',
 'DE Resolutien gisteren genomen',
 'DE NEON g ae ien genomen',
 'DE Re oren Ii B NESen ge15',
 'DE Resolutien voor eergil',
 'DE Resolutien voor eergisteren genoomen',
 'DE Resolutien voorsreede Donderdagh genoomen',
 'EAEAAER suri ee gin eren genoomen',
 'ESWARAR uyt de Provintie van Gelderland',
 'Een extraorsinaris Gedeputeerde uyt de Prowimtie van Overyssel',
 'Gelderlund',
 'Hp» 1741 van Cattenburch',
 'Jovis den 5 November vier en veertigh',
 'Leen',
 'Lunae den 14 September1744PRAESIIDE',
 'Lunae den 25 Mey1744',
 'Martis den 26 Mey1744',
 'met',
 'PRAAESEHENTIPUS',
 'PRADESENTIEBUS',
 'PRAESITDE',
 'PRAEZE

In [81]:
non_suspect = [k for k in rconsolidated_groups if k not in rsuspected_items]

In [82]:
nsr = [{k:herensearcher.find_matches(k)} for k in non_suspect]

In [84]:
rconsolidated_groups

{'met vyf': ['met eén',
  'met eenu',
  'met vwys',
  'met vys',
  'met cen',
  "'met vyf",
  'met wyf',
  'met ses'],
 '17144 DE Resosutien gisteren genomen': ['1744 DE Resorditieen ginteren geno men',
  'OverysselDE Resolutien gisteren genomen',
  'OverWESDE Resoluten gisteren genomen'],
 '1734 Bentinck': ['1744 Bentinck', '1744Bentinck'],
 '1744 Bergsma': [],
 '1744 Heeren Gecommitteerde Raaden van de pypvintie van Holland en Westvriesland': [],
 '1744 Vun Utenhove tot Boitestein': [],
 '1744 neevens die welcke gevoeght is geweest by een Missive van den Secretaris Bolch': [],
 '1744 plint van meeninge was': [],
 'Ingilby': ['Ingiloyv',
  'lrgilby',
  'JIngelby',
  'lr gzilby',
  'JIngilby',
  'si gilby',
  'Ingelby',
  'Ingelly',
  'Ingelhy',
  'Ingilty',
  'TIngelby',
  'In gelby',
  'Iugelby',
  'gilby',
  'Inilby',
  '1ngilby',
  'Ingilôy',
  'Inzilby',
  'h gilby'],
 'A G': [],
 'Bergsma van lisma': [],
 'Holland en Westvriesland': ['Holland en Yestvriesland',
  'Bolland en Wees

In [ ]:
non_suspect